# Objective: Fine-tune GPT-2 with the resulting filtered dataset

### 1. Imports and Global Settings

In [1]:
from datasets import load_dataset, disable_caching
from transformers import GPT2TokenizerFast, DataCollatorWithPadding, GPT2ForSequenceClassification, set_seed
import torch
from torch.nn.functional import one_hot
import sys
sys.path.append('..')
from utils_ import tokenize, train_classifier
import pickle
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)
disable_caching()

In [2]:
# set up tokeniser
# padding to left because GPT2 uses last token for prediction
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2-medium", padding_side = 'left', \
                                              padding = True, truncation = True)
tokenizer.pad_token = tokenizer.eos_token # pad with 'eos' token

In [3]:
# set up data collator - https://huggingface.co/docs/transformers/main_classes/data_collator
# this is a (callable) helper object that sends batches of data to the model
data_collator = DataCollatorWithPadding(tokenizer, padding = 'max_length', \
                                         return_tensors = 'pt', max_length = 128)

In [4]:
# read in indices removed by AFLite for each seed
with open('removed_idx.pkl', 'rb') as f:
    removed = pickle.load(f)

## 2. Pre-processing Routine for SNLI
- Get SNLI Dataset (Train fold) and shuffle it using the same seed as used for obtaining GPT-2 based Feature Representation (see notebook [Filtering_Part1.ipynb](https://github.com/shashiniyer/adversarial_nli_gpt2/blob/main/gpt2-medium/notebooks_and_scripts/Filtering_Part1.ipynb))
- Remove instances without gold standard labels, i.e., label = -1
- One-hot encoding for labels
- Partition data 10%/90%; use the 90% as unfiltered `train`
- Filter out what AFLite run removed (see notebook [Filtering_Part2.ipynb](https://github.com/shashiniyer/adversarial_nli_gpt2/blob/main/gpt2-medium/notebooks_and_scripts/Filtering_Part2.ipynb))
- Tokenise

In [5]:
def preprocess_snli(seed):
    
    # set up `train` dataset
    snli_train = load_dataset('snli', split = 'train').shuffle(seed = 42)
    snli_train = snli_train.filter(lambda x: x['label'] != -1).map( \
        lambda x: {'label': one_hot(torch.tensor(x['label']), 3).type(torch.float32).numpy()}, \
        batched = True)
    train = snli_train.select(range(int(len(snli_train)/10), len(snli_train)))
    
    # filter out what AFLite run removed
    removed_idx = [int(x) for x in removed[seed].split(',')[1:]] 
    train = train.select(set(range(len(train))) - set(removed_idx))
    
    # tokenize data
    train = train.map(lambda x: tokenize(tokenizer, x['premise'] + '|' + x['hypothesis']))
    len_bef_exclusion = len(train)

    # exclude instances with > 128 tokens
    train = train.filter(lambda x: x['exclude'] == False)
    len_aft_exclusion = len(train)

    # print message if instances were in fact excluded
    if len_bef_exclusion - len_aft_exclusion > 0:

        print(f'{len_bef_exclusion - len_aft_exclusion} ' + \
              f'({(len_bef_exclusion/len_aft_exclusion - 1)*100:>2f}%) sequences excluded')

    # keep only needed columns, set data format to PyTorch
    train.set_format(type = 'torch', columns = ['label', 'input_ids', 'attention_mask'])

    return(train)

### 3. Run fine-tuning step and save resulting models

In [6]:
# hyper-parameters for model training
batch_size = 32 # constrained by GPU memory
lr = 1e-5 # set to match Le et al. (2020) - https://arxiv.org/abs/2002.04108

# from Filtering_Part2.ipynb
AFLite_seeds = [0, 1, 2, 3, 4]
    
# Begin fine-tuning
for seed in AFLite_seeds:
    
    # print log message
    print(f'Seed {seed} begin')
    
    # read in filtered data
    data = preprocess_snli(seed)
    print('Filtered Data Read In')
    
    # instantiate new GPT-2 based model
    model = GPT2ForSequenceClassification.from_pretrained("gpt2-medium", 
                                      num_labels=3,
                                      problem_type="multi_label_classification")
    model.config.pad_token_id = model.config.eos_token_id # specify pad_token used by tokenizer
    
    # set up data loader
    dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, \
                                             shuffle=True, collate_fn=data_collator)
    
    # set up optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)    
    
    # fine-tune model
    torch.save(train_classifier(model, dataloader, optimizer, device), \
              'AFLite_fine_tuned_model_seed' + str(seed) + '.pth')
    
    # print log message
    print(f'Seed {seed} complete')    

Seed 4 begin


Reusing dataset snli (/home/shana92/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

  0%|          | 0/266415 [00:00<?, ?ex/s]

...

loss: 1.129973  [    0/266415]
loss: 0.599545  [26624/266415]
loss: 0.614978  [53248/266415]
loss: 0.582858  [79872/266415]
loss: 0.525758  [106496/266415]
loss: 0.486906  [133120/266415]
loss: 0.357164  [159744/266415]
loss: 0.394921  [186368/266415]
loss: 0.448161  [212992/266415]
loss: 0.279881  [239616/266415]
loss: 0.246570  [266240/266415]
Epoch average loss: 0.48667386174201965


  0%|          | 0/8326 [00:00<?, ?it/s]

loss: 0.428860  [    0/266415]
loss: 0.449402  [26624/266415]
loss: 0.171800  [53248/266415]
loss: 0.322920  [79872/266415]
loss: 0.339696  [106496/266415]
loss: 0.270904  [133120/266415]
loss: 0.248552  [159744/266415]
loss: 0.290512  [186368/266415]
loss: 0.236461  [212992/266415]
loss: 0.232649  [239616/266415]
loss: 0.347984  [266240/266415]
Epoch average loss: 0.31907910108566284


  0%|          | 0/8326 [00:00<?, ?it/s]

loss: 0.184027  [    0/266415]
loss: 0.176777  [26624/266415]
loss: 0.263505  [53248/266415]
loss: 0.275502  [79872/266415]
loss: 0.240916  [106496/266415]
loss: 0.298550  [133120/266415]
loss: 0.280404  [159744/266415]
loss: 0.195512  [186368/266415]
loss: 0.262465  [212992/266415]
loss: 0.175238  [239616/266415]
loss: 0.320463  [266240/266415]
Epoch average loss: 0.27321189641952515
Done!
Seed 4 complete
